# Bepub V.1.0.0

In [1]:
# 1. ติดตั้ง Library ที่จำเป็น
!pip install -q typhoon-ocr ebooklib unicodedata2 pdfplumber PyPDF2 markdown
!apt-get install -y poppler-utils > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 11.2 MB/s eta 0:00:00


In [2]:
import os
import re
import unicodedata
import zipfile
import markdown
import shutil
import gradio as gr
from PyPDF2 import PdfReader, PdfWriter
from typhoon_ocr import ocr_document

is_cancelled = False

def cancel_process():
    global is_cancelled
    is_cancelled = True
    return "🛑 กำลังหยุดการทำงานและเตรียมไฟล์ล่าสุด..."

def process_and_build_epub(api_key, pdf_file, start_page, end_page, custom_remove, progress=gr.Progress()):
    global is_cancelled
    is_cancelled = False

    if not api_key or not pdf_file:
        return None, None, "❌ กรุณาตรวจสอบ API Key และไฟล์ PDF"

    s_p = int(start_page)
    e_p = int(end_page)

    input_path = pdf_file.name
    full_raw_name = os.path.basename(input_path)
    base_name = full_raw_name.rsplit('.', 1)[0]

    recovery_txt = "temp_recovery_content.txt"
    if os.path.exists(recovery_txt): os.remove(recovery_txt)

    full_text_content = ""

    try:
        os.environ["TYPHOON_OCR_API_KEY"] = api_key
        reader = PdfReader(input_path)
        total_in_pdf = len(reader.pages)
        actual_end = total_in_pdf if (e_p == 0 or e_p > total_in_pdf) else e_p
        target_indices = [i - 1 for i in range(s_p, actual_end + 1)]

        for idx in progress.tqdm(target_indices, desc="กำลัง OCR"):
            if is_cancelled: break

            p_num = idx + 1
            temp_pdf = f"temp_page_{p_num}.pdf"
            writer = PdfWriter()
            writer.add_page(reader.pages[idx])
            with open(temp_pdf, "wb") as f: writer.write(f)

            try:
                page_res = ocr_document(temp_pdf)
                if page_res:
                    # ใช้ 'NFC' เพื่อให้วรรณยุกต์รวมร่างกับตัวอักษร ไม่แยกชั้น
                    text = unicodedata.normalize('NFC', str(page_res))
                    full_text_content += text + "\n\n--- Page Break ---\n\n"
                    with open(recovery_txt, "a", encoding="utf-8-sig") as f:
                        f.write(text + "\n\n--- Page Break ---\n\n")
            finally:
                if os.path.exists(temp_pdf): os.remove(temp_pdf)

        status_final = "✅ สำเร็จ!" if not is_cancelled else "🛑 หยุดโดยผู้ใช้"

        # ส่ง base_name ดั้งเดิมไปสร้างไฟล์
        txt_path, epub_path, _ = final_build(full_text_content, base_name)
        return txt_path, epub_path, status_final

    except Exception as e:
        if full_text_content:
            txt_path, epub_path, _ = final_build(full_text_content, f"RECOVERED_{base_name}")
            return txt_path, epub_path, f"⚠️ Error: {str(e)}"
        return None, None, f"❌ Error: {str(e)}"

def final_build(content, base_name):
    if not content.strip(): return None, None, "⚠️ ไม่มีเนื้อหา"

    # คลีนเนื้อหาเบื้องต้น
    clean = re.sub(r'\n\*\s*\d+\s*\n|\*\s*\d+\s*\*|\'', '', content)
    clean = re.sub(r'\n\s*\n', '\n\n', clean)
    sections = clean.split('--- Page Break ---')

    build_dir = "epub_build_temp"
    if os.path.exists(build_dir): shutil.rmtree(build_dir)
    os.makedirs(f'{build_dir}/OEBPS', exist_ok=True)
    os.makedirs(f'{build_dir}/META-INF', exist_ok=True)

    with open(f'{build_dir}/mimetype', 'w') as f: f.write('application/epub+zip')
    with open(f'{build_dir}/META-INF/container.xml', 'w') as f:
        f.write('<?xml version="1.0"?><container version="1.0" xmlns="urn:oasis:names:tc:opendocument:xmlns:container"><rootfiles><rootfile full-path="OEBPS/content.opf" media-type="application/oebps-package+xml"/></rootfiles></container>')

    manifest, spine = "", ""
    for i, sec in enumerate(sections):
        if not sec.strip(): continue
        html = markdown.markdown(sec.strip())
        f_name = f'p{i}.xhtml'
        with open(f'{build_dir}/OEBPS/{f_name}', 'w', encoding='utf-8') as f:
            f.write(f'<html><body>{html}</body></html>')
        manifest += f'<item id="p{i}" href="{f_name}" media-type="application/xhtml+xml"/>\n'
        spine += f'<itemref idref="p{i}"/>\n'

    # ใน content.opf ใช้ base_name ดั้งเดิมเป็น Title
    with open(f'{build_dir}/OEBPS/content.opf', 'w', encoding='utf-8') as f:
        f.write(f'<?xml version="1.0"?><package xmlns="http://www.idpf.org/2007/opf" version="2.0"><metadata xmlns:dc="http://purl.org/dc/elements/1.1/"><dc:title>{base_name}</dc:title><dc:language>th</dc:language></metadata><manifest>{manifest}<item id="ncx" href="toc.ncx" media-type="application/x-dtbncx+xml"/></manifest><spine toc="ncx">{spine}</spine></package>')

    with open(f'{build_dir}/OEBPS/toc.ncx', 'w', encoding='utf-8') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?><ncx xmlns="http://www.daisy.org/z3986/2005/ncx/" version="2005-1"><navMap><navPoint id="n1" playOrder="1"><navLabel><text>Start</text></navLabel><content src="p0.xhtml"/></navPoint></navMap></ncx>')

    # กำหนดชื่อไฟล์ output
    epub_name = unicodedata.normalize("NFC", f"{base_name}.epub")
    txt_name = unicodedata.normalize("NFC", f"{base_name}.txt")

    # บันทึกไฟล์ TXT
    with open(txt_name, "w", encoding="utf-8-sig") as f:
        f.write(content)

    # แพ็คไฟล์ ZIP
    with zipfile.ZipFile(epub_name, 'w', zipfile.ZIP_DEFLATED) as z:
        for r, d, fs in os.walk(build_dir):
            for file in fs:
                file_path = os.path.join(r, file)
                z.write(file_path, os.path.relpath(file_path, build_dir))

    shutil.rmtree(build_dir)
    return txt_name, epub_name, "✅"

# --- UI Interface ---
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="📂 1. Upload PDF", file_types=[".pdf"])
        with gr.Column(scale=1):
            with gr.Group():
                gr.Markdown("### ⚙️ Configs")
                api_input = gr.Textbox(label="API Key", type="password")
                with gr.Row():
                    start_in = gr.Number(label="เริ่มหน้า", value=1, precision=0)
                    end_in = gr.Number(label="สิ้นสุด", value=1000, precision=0)
                custom_remove = gr.Textbox(label="ลบคำเฉพาะ (ใช้ , คั่น)")

    with gr.Row():
        btn_start = gr.Button("🚀 เริ่มทำงาน", variant="primary", scale=1)
        btn_stop = gr.Button("🛑 หยุดทันที", variant="stop", scale=1)

    with gr.Group():
        gr.Markdown("### 📥 Download Results")
        status_msg = gr.Textbox(label="Status", interactive=False)
        with gr.Row():
            epub_output = gr.File(label="ไฟล์ EPUB")
            txt_output = gr.File(label="ไฟล์ TXT")

    # สั่งเริ่มทำงาน
    btn_start.click(
        fn=process_and_build_epub,
        inputs=[api_input, file_input, start_in, end_in, custom_remove],
        outputs=[txt_output, epub_output, status_msg]
    )

    # หากสั่งหยุด
    btn_stop.click(fn=cancel_process, outputs=status_msg)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54b33f69779fb738b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
